# Pickup to Delivery Overall

In [1]:
import os
import sys
import datetime
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np

sys.path.insert(0, os.path.expanduser('./'))
import query_runner as qr
import utils
from estimator import BaselineModel_sum, BaselineModel_mean, LinearModel, RegressionTreeMethod

In [2]:
dwh_config, livedb_config, parameters_config = utils.load_config(config_file='./config.ini')
start_date = parameters_config['start_date']
end_date = parameters_config['end_date']
country_code = parameters_config['country_code']
cities = parameters_config['cities']

print(f'Start date: {start_date} | End date: {end_date} | Countries: {country_code} | Cities: {cities}')

Start date: 2024-09-30 | End date: 2024-10-20 | Countries: ES | Cities: 'MAD', 'BCN', 'SEV', 'ALC'


## Load the dataset

In [3]:
data = pd.read_parquet("data/parquet/dataframe.parquet")

## Hyperparameters

In [4]:
test_set_perc = 0.1
days_for_test = 7
k_cv = 5

## Database split

As we have partitioned the data by city and creation date, we can use this information to split the data. This will help to avoid data leakage, as we will not have data from the future in the training set.
This is much better than just sorting the data by the creation timestamp and taking 10% of the dataset as test set, as we did before.

In [5]:
# We take the last week of the dataset to test the model
begin_test_date = pd.to_datetime(end_date) - pd.Timedelta(days=days_for_test-1)
begin_test_date = begin_test_date.strftime("%Y-%m-%d")
print(f'Start date: {start_date} | Begin test date: {begin_test_date} | End date: {end_date}')

Start date: 2024-09-30 | Begin test date: 2024-10-14 | End date: 2024-10-20


In [6]:
data_train = pd.read_parquet("data/parquet/dataframe.parquet/", filters=[('creation_date', '<', begin_test_date)])
data_train.head()

,country_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,delivery_timestamp,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,creation_date,city_code
0,ES,100901465337,169809943,2024-09-30 21:45:07+00:00,2024-09-30 21:45:08+00:00,MOTORBIKE,38.36,-0.49,38.38,-0.49,...,2024-09-30 22:10:01+00:00,0.67,-0.01,0.67,-0.01,2355.55,2355.55,2561.90,2024-09-30,ALC
1,ES,100901489101,9516729,2024-09-30 21:55:46+00:00,2024-09-30 21:55:47+00:00,CAR,38.37,-0.42,38.36,-0.42,...,2024-09-30 22:09:18+00:00,0.67,-0.01,0.67,-0.01,673.66,673.66,903.52,2024-09-30,ALC
2,ES,100899913152,172855743,2024-09-30 10:30:55+00:00,2024-09-30 10:30:57+00:00,MOTORBIKE,38.35,-0.49,38.35,-0.49,...,2024-09-30 10:49:56+00:00,0.67,-0.01,0.67,-0.01,614.69,614.69,622.46,2024-09-30,ALC
3,ES,100900447439,170201413,2024-09-30 14:46:15+00:00,2024-09-30 14:46:16+00:00,CAR,38.37,-0.47,38.36,-0.44,...,2024-09-30 15:11:36+00:00,0.67,-0.01,0.67,-0.01,2962.48,2962.48,3332.04,2024-09-30,ALC
4,ES,100900529830,176424631,2024-09-30 15:22:32+00:00,2024-09-30 15:22:33+00:00,MOTORBIKE,38.35,-0.48,38.35,-0.50,...,2024-09-30 15:56:07+00:00,0.67,-0.01,0.67,-0.01,1708.76,1708.76,1711.24,2024-09-30,ALC


In [7]:
# Check that there are no nulls deriving from a wrong writing of parquet files (appending instead of overwriting)
data_train.isnull().sum().sum()

np.int64(0)

We will only use the feature `activation_timestamp`, as we are simulating being Glovo Jarvis engine that has to estimate the PDO time in order to decide to assign an order to a specific courier. The column `creation_timestamp` is threfore redundant and we will not include it in the model.

In [8]:
train_columns = ['country_code', 'city_code', 'order_id', 'courier_id', 'activation_timestamp', 'transport', 'pickup_latitude', 'pickup_longitude',
                 'delivery_latitude', 'delivery_longitude', 'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad', 'delivery_latitude_rad',
                 'delivery_longitude_rad', 'pd_distance_haversine_m', 'pd_distance_haversine_m_sk', 'pd_distance_manhattan_m']
X_train = data_train[train_columns].copy()
X_train.head()

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,time_zone,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m
0,ES,ALC,100901465337,169809943,2024-09-30 21:45:08+00:00,MOTORBIKE,38.36,-0.49,38.38,-0.49,Europe/Madrid,0.67,-0.01,0.67,-0.01,2355.55,2355.55,2561.90
1,ES,ALC,100901489101,9516729,2024-09-30 21:55:47+00:00,CAR,38.37,-0.42,38.36,-0.42,Europe/Madrid,0.67,-0.01,0.67,-0.01,673.66,673.66,903.52
2,ES,ALC,100899913152,172855743,2024-09-30 10:30:57+00:00,MOTORBIKE,38.35,-0.49,38.35,-0.49,Europe/Madrid,0.67,-0.01,0.67,-0.01,614.69,614.69,622.46
3,ES,ALC,100900447439,170201413,2024-09-30 14:46:16+00:00,CAR,38.37,-0.47,38.36,-0.44,Europe/Madrid,0.67,-0.01,0.67,-0.01,2962.48,2962.48,3332.04
4,ES,ALC,100900529830,176424631,2024-09-30 15:22:33+00:00,MOTORBIKE,38.35,-0.48,38.35,-0.50,Europe/Madrid,0.67,-0.01,0.67,-0.01,1708.76,1708.76,1711.24


In [9]:
y_train = (data_train['delivery_timestamp'] - data_train['pickup_timestamp']).dt.total_seconds()
y_train = pd.Series(y_train, name='pickup_to_delivery')
y_train.head()

0   564.31
1   464.86
2   511.03
3   906.38
4   680.04
Name: pickup_to_delivery, dtype: float64

In [10]:
y_train.isnull().sum().sum()

np.int64(0)

In [11]:
data_test = pd.read_parquet("data/parquet/dataframe.parquet", filters=[('creation_date', '>=', begin_test_date)])
data_test.head()

,country_code,order_id,courier_id,creation_timestamp,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,delivery_timestamp,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,creation_date,city_code
0,ES,100933187613,176241347,2024-10-14 23:24:53+00:00,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,2024-10-14 23:45:58+00:00,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,2024-10-14,ALC
1,ES,100933198445,10191824,2024-10-14 23:37:45+00:00,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,2024-10-15 00:04:30+00:00,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,2024-10-14,ALC
2,ES,100931749423,169099229,2024-10-14 12:57:37+00:00,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,2024-10-14 13:22:05+00:00,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,2024-10-14,ALC
3,ES,100931867858,177428955,2024-10-14 13:44:05+00:00,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,2024-10-14 14:26:28+00:00,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,2024-10-14,ALC
4,ES,100931917343,3548605,2024-10-14 14:03:19+00:00,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,2024-10-14 14:28:00+00:00,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,2024-10-14,ALC


In [12]:
X_test = data_test[train_columns].copy()
X_test.head()

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,time_zone,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,Europe/Madrid,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,Europe/Madrid,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,Europe/Madrid,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,Europe/Madrid,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,Europe/Madrid,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43


In [13]:
X_test.isnull().sum().sum()

np.int64(0)

In [14]:
y_test = (data_test['delivery_timestamp'] - data_test['pickup_timestamp']).dt.total_seconds()
y_test = pd.Series(y_test, dtype=np.float64, name='pickup_to_delivery')
y_test.head()

0    319.68
1    811.65
2    614.70
3   1111.15
4    806.57
Name: pickup_to_delivery, dtype: float64

In [15]:
y_test.isnull().sum().sum()

np.int64(0)

In [16]:
print("Train datasets shapes: ", X_train.shape, y_train.shape)
print("Test datasets shapes: ", X_test.shape, y_test.shape)

Train datasets shapes:  (688815, 18) (688815,)
Test datasets shapes:  (354799, 18) (354799,)


## Baseline Models

### BaselineModel_sum

In [17]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:27:32.154570


In [18]:
model_bl_sum = BaselineModel_sum()
model_bl_sum.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 18), y: (688815,)


In [19]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_bl_sum_time = end - start
print(f"Time elapsed: {model_bl_sum_time}")

End time: 2025-06-14 17:27:32.619463
Time elapsed: 0:00:00.464893


In [20]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_bl_sum.predict(X_test.iloc[0]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 345.14675182584176
Diff: 25.46975182584174


In [21]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_bl_sum.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,345.15,25.47
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,344.10,-467.55
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,862.35,247.65
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,1425.92,314.76
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,266.35,-540.22
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,2713.04,1607.68
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,364.35,73.80
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,1097.86,343.07
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,855.01,-621.79
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,916.85,294.82


### Evaluation pipeline

In [22]:
model_bl_sum_eval = model_bl_sum.evaluate(X_test, y_test)
model_bl_sum_eval

y_hat <class 'pandas.core.series.Series'> float64
y_test <class 'pandas.core.series.Series'> float64


(np.float64(256.3865388687307), np.float64(179628.8794954057))

### BaselineModel_mean

In [23]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:06.795664


In [24]:
model_bl_mean = BaselineModel_mean()
model_bl_mean.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 18), y: (688815,)


In [25]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_bl_mean_time = end - start
print(f"Time elapsed: {model_bl_mean_time}")

End time: 2025-06-14 17:28:07.027515
Time elapsed: 0:00:00.231851


In [26]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_bl_mean.predict(X_test.iloc[0]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 292.3004304172709
Diff: -27.376569582729132


In [27]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_bl_mean.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,292.30,-27.38
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,281.54,-530.11
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,659.83,45.13
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,1207.59,96.44
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,203.80,-602.77
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,2219.77,1114.41
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,298.11,7.55
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,840.03,85.24
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,724.09,-752.71
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,750.15,128.12


### Evaluation pipeline

In [28]:
model_bl_mean_eval = model_bl_mean.evaluate(X_test, y_test)
model_bl_mean_eval

(np.float64(239.81356923333743), np.float64(157993.39824654086))

## Linear Models

In [29]:
# Train on a small subset to check if the model is working
X_train_smaller = X_train.head(1000)
y_train_smaller = y_train.head(1000)

### Linear Model encoding dummy variables

In [30]:
model_linear_smaller = LinearModel(model_type='linear', encoding='dummy')
model_linear_smaller.fit(X_train_smaller, y_train_smaller)

INFO:root:Train datasets shapes: X: (1000, 19), y: (1000,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (1000, 26)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [31]:
y_test_0 = y_train_smaller.loc[0]
y_test_0_pred = model_linear_smaller.predict(X_train_smaller.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 564.308
Predicted delivery time: 781.9825627692044
Diff: 217.67456276920439


#### Train on 3 days of data

In [32]:
# We take the last week of the dataset to test the model
days_for_train = 3
days_for_test = 1
end_train_date = pd.to_datetime(start_date) + pd.Timedelta(days=days_for_train)
end_train_date = end_train_date.strftime("%Y-%m-%d")
print(f'Start date: {start_date} | End test date: {end_train_date} | Test date: {end_date}')

Start date: 2024-09-30 | End test date: 2024-10-03 | Test date: 2024-10-20


In [33]:
data_train_3d = pd.read_parquet("data/parquet/dataframe.parquet/", filters=[('creation_date', '<', end_train_date)])
X_train_3d = data_train_3d[train_columns].copy()
y_train_3d = y_train[X_train_3d.index]
y_train_3d = pd.Series(y_train_3d, name='pickup_to_delivery')
data_test_3d = pd.read_parquet("data/parquet/dataframe.parquet", filters=[('creation_date', '=', end_date)])
X_test_3d = data_test_3d[train_columns].copy()
y_test_3d = y_test[X_test_3d.index]
y_test_3d = pd.Series(y_test_3d, dtype=np.float64, name='pickup_to_delivery')
print("Train datasets 3d shapes: ", X_train_3d.shape, y_train_3d.shape)
print("Test datasets 3d shapes: ", X_test_3d.shape, y_test_3d.shape)

Train datasets 3d shapes:  (132533, 18) (132533,)
Test datasets 3d shapes:  (56857, 18) (56857,)


In [34]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:39.539319


In [35]:
model_linear_3d = LinearModel(model_type='linear', encoding='dummy')
model_linear_3d.fit(X_train_3d, y_train_3d)

INFO:root:Train datasets shapes: X: (132533, 18), y: (132533,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (132533, 26)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [36]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_3d_time = end - start
print(f"Time elapsed: {model_linear_3d_time}")

End time: 2025-06-14 17:28:39.773748
Time elapsed: 0:00:00.234429


In [37]:
model_linear_3d_eval = model_linear_3d.evaluate(X_test_3d, y_test_3d)
model_linear_3d_eval

(np.float64(4850.462204065004), np.float64(23788905.201602723))

#### Train on a week of data

In [38]:
# We take the last week of the dataset to test the model
days_for_train = 7
days_for_test = 3
end_train_date = pd.to_datetime(start_date) + pd.Timedelta(days=days_for_train)
end_train_date = end_train_date.strftime("%Y-%m-%d")
begin_test_date = pd.to_datetime(end_date) - pd.Timedelta(days=days_for_test-1)
begin_test_date = begin_test_date.strftime("%Y-%m-%d")
print(f'Start date: {start_date} | End test date: {end_train_date} | Begin test date: {begin_test_date} | End date: {end_date}')

Start date: 2024-09-30 | End test date: 2024-10-07 | Begin test date: 2024-10-18 | End date: 2024-10-20


In [39]:
data_train_7d = pd.read_parquet("data/parquet/dataframe.parquet/", filters=[('creation_date', '<', end_train_date)])
X_train_7d = data_train_7d[train_columns].copy()
y_train_7d = y_train[X_train_7d.index]
y_train_7d = pd.Series(y_train_7d, name='pickup_to_delivery')
data_test_7d = pd.read_parquet("data/parquet/dataframe.parquet", filters=[('creation_date', '>=', begin_test_date)])
X_test_7d = data_test_7d[train_columns].copy()
y_test_7d = y_test[X_test_7d.index]
y_test_7d = pd.Series(y_test_7d, dtype=np.float64, name='pickup_to_delivery')
print("Train datasets 7d shapes: ", X_train_7d.shape, y_train_7d.shape)
print("Test datasets 7d shapes: ", X_test_7d.shape, y_test_7d.shape)

Train datasets 7d shapes:  (350485, 18) (350485,)
Test datasets 7d shapes:  (171489, 18) (171489,)


In [40]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:40.187849


In [41]:
model_linear_7d = LinearModel(model_type='linear', encoding='dummy')
model_linear_7d.fit(X_train_7d, y_train_7d)

INFO:root:Train datasets shapes: X: (350485, 18), y: (350485,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (350485, 26)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [42]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_7d_time = end - start
print(f"Time elapsed: {model_linear_7d_time}")

End time: 2025-06-14 17:28:40.863942
Time elapsed: 0:00:00.676093


In [43]:
model_linear_7d_eval = model_linear_7d.evaluate(X_test_7d, y_test_7d)
model_linear_7d_eval

(np.float64(300.4289466948376), np.float64(198128.49677930487))

#### Train on full data

In [44]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:41.047768


In [45]:
model_linear = LinearModel(model_type='linear', encoding='dummy')
model_linear.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [46]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_time = end - start
print(f"Time elapsed: {model_linear_time}")

End time: 2025-06-14 17:28:42.251810
Time elapsed: 0:00:01.204042


In [47]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_linear.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,425.07,105.39
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,413.37,-398.27
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,731.36,116.66
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,894.00,-217.16
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,399.77,-406.79
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,1540.69,435.33
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,437.36,146.80
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,861.22,106.43
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,628.16,-848.64
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,694.49,72.46


In [48]:
model_linear_eval = model_linear.evaluate(X_test, y_test)
model_linear_eval

(np.float64(183.02056888939788), np.float64(114183.02259706463))

### LinearModel cyclical encoding

In [49]:
model_linear_cyclical_smaller = LinearModel(model_type='linear', encoding='cyclical')
model_linear_cyclical_smaller.fit(X_train_smaller, y_train_smaller)

INFO:root:Train datasets shapes: X: (1000, 19), y: (1000,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (1000, 33)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [50]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_linear_cyclical_smaller.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: -2221.862971879542
Diff: -2541.539971879542


#### Train on full data

In [51]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:43.138885


In [52]:
model_linear_cyclical = LinearModel(model_type='linear', encoding='cyclical')
model_linear_cyclical.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 34)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [53]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_cyclical_time = end - start
print(f"Time elapsed: {model_linear_cyclical_time}")

End time: 2025-06-14 17:28:45.436741
Time elapsed: 0:00:02.297856


In [54]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_linear_cyclical.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,384.36,64.68
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,370.80,-440.85
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,734.68,119.98
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,899.01,-212.15
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,402.98,-403.58
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,1550.03,444.67
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,437.06,146.50
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,847.54,92.75
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,602.05,-874.75
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,661.98,39.94


In [55]:
model_linear_cyclical_eval = model_linear_cyclical.evaluate(X_test, y_test)
model_linear_cyclical_eval

(np.float64(183.28648174613807), np.float64(122189.95765097688))

### Linear Model with normalization

Compare performance of the model with and without normalizatio or standardization.

With the `minmax` option, the scaled data has zero mean and unit variance.

In [56]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:47.168405


In [57]:
model_linear_minmaxscaler = LinearModel(model_type='linear', encoding='dummy', standardize='minmax')
model_linear_minmaxscaler.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [58]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_minmaxscaler_time = end - start
print(f"Time elapsed: {model_linear_minmaxscaler_time}")

End time: 2025-06-14 17:28:49.081700
Time elapsed: 0:00:01.913295


In [59]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_linear_minmaxscaler.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 425.1353639401541
Diff: 105.45836394015407


In [60]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_linear_minmaxscaler.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,425.15,105.47
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,413.28,-398.37
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,732.25,117.55
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,893.84,-217.32
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,399.78,-406.78
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,1540.60,435.24
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,437.49,146.93
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,861.21,106.41
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,628.23,-848.57
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,694.34,72.31


In [61]:
model_linear_minmaxscaler_eval = model_linear_minmaxscaler.evaluate(X_test, y_test)
model_linear_minmaxscaler_eval

(np.float64(183.01388679235822), np.float64(114181.34344234352))

### Linear Model with standardization

With the `stdscaler` option, the scaled data is transformed to have values from 0 and 1.

In [62]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:50.600780


In [63]:
model_linear_stdscaler = LinearModel(model_type='linear', encoding='dummy', standardize='stdscaler')
model_linear_stdscaler.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [64]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_stdscaler_time = end - start
print(f"Time elapsed: {model_linear_stdscaler_time}")

End time: 2025-06-14 17:28:52.583798
Time elapsed: 0:00:01.983018


In [65]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_linear_stdscaler.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 423.946566660288
Diff: 104.26956666028798


In [66]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_linear_stdscaler.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,423.95,104.27
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,413.23,-398.42
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,725.09,110.39
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,893.88,-217.27
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,401.49,-405.07
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,1530.51,425.14
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,437.29,146.73
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,856.45,101.66
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,627.49,-849.32
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,689.57,67.54


In [67]:
model_linear_stdscaler_eval = model_linear_stdscaler.evaluate(X_test, y_test)
model_linear_stdscaler_eval

(np.float64(183.04983268667368), np.float64(114183.72156898087))

### Linear Model SGD encoding dummy variables

#### Train on full data

In [68]:
start = datetime.datetime.now()
print(f"Start time: {start}")

Start time: 2025-06-14 17:28:54.059406


In [69]:
model_linear_SGD_stdscaler = LinearModel(model_type='sgd', encoding='dummy', standardize='stdscaler')
model_linear_SGD_stdscaler.fit(X_train, y_train)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [70]:
end = datetime.datetime.now()
print(f"End time: {end}")
model_linear_SGD_stdscaler_time = end - start
print(f"Time elapsed: {model_linear_SGD_stdscaler_time}")

End time: 2025-06-14 17:28:57.695197
Time elapsed: 0:00:03.635791


In [71]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_linear_SGD_stdscaler.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 413.10175802730805
Diff: 93.42475802730803


In [72]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_linear_SGD_stdscaler.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,413.10,93.42
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,399.41,-412.24
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,734.49,119.79
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,841.13,-270.02
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,394.73,-411.83
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,1525.76,420.39
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,428.66,138.10
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,830.23,75.44
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,604.77,-872.03
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,651.43,29.40


In [73]:
model_linear_SGD_stdscaler_eval = model_linear_SGD_stdscaler.evaluate(X_test, y_test)
model_linear_SGD_stdscaler_eval

(np.float64(182.8751370869995), np.float64(115406.52654512945))

## Tree Models

### Decision Tree

In [74]:
start = datetime.datetime.now()
model_decision_tree = RegressionTreeMethod(model_type = 'tree', encoding = 'dummy')
model_decision_tree.fit(X_train, y_train)
end = datetime.datetime.now()
model_decision_tree_time = end - start
model_decision_tree_eval = model_decision_tree.evaluate(X_test, y_test)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [95]:
start = datetime.datetime.now()
model_decision_tree_cyclical = RegressionTreeMethod(model_type = 'tree', encoding = 'cyclical')
model_decision_tree_cyclical.fit(X_train, y_train)
end = datetime.datetime.now()
model_decision_tree_cyclical_time = end - start
print(f"Time elapsed: {model_decision_tree_cyclical_time}")

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables


INFO:root:Encoded dataset shape: X: (688815, 34)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


Time elapsed: 0:00:29.306296


In [76]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_decision_tree_cyclical.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 10366.521
Diff: 10046.844000000001


In [77]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_decision_tree_cyclical.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,10366.52,10046.84
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,18256.90,17445.25
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,15136.67,14521.97
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,15136.67,14025.52
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,18256.90,17450.33
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,15136.67,14031.31
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,18256.90,17966.34
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,15136.67,14381.88
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,15136.67,13659.87
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,15136.67,14514.64


In [78]:
model_decision_tree_cyclical_eval = model_decision_tree_cyclical.evaluate(X_test, y_test)
model_decision_tree_cyclical_eval

(np.float64(14516.32596154724), np.float64(231749303.6833687))

### Random Forest

In [79]:
start = datetime.datetime.now()
model_random_forest = RegressionTreeMethod(model_type = 'randomforest', encoding = 'dummy', n_estimators = 100)
model_random_forest.fit(X_train, y_train)
end = datetime.datetime.now()
model_random_forest_time = end - start
model_random_forest_eval = model_random_forest.evaluate(X_test, y_test)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [80]:
start = datetime.datetime.now()
model_random_forest_cyclical = RegressionTreeMethod(model_type = 'randomforest', encoding = 'cyclical', n_estimators = 100)
model_random_forest_cyclical.fit(X_train, y_train)
end = datetime.datetime.now()
model_random_forest_cyclical_time = end - start
print(f"Time elapsed: {model_random_forest_cyclical_time}")

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 34)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


Time elapsed: 0:21:03.272824


In [81]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_random_forest_cyclical.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 11757.951929278683
Diff: 11438.274929278683


In [82]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_random_forest_cyclical.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,11757.95,11438.27
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,20429.60,19617.95
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,12499.68,11884.98
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,12524.44,11413.28
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,17040.77,16234.21
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,12502.40,11397.04
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,20459.97,20169.42
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,12549.89,11795.10
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,12521.58,11044.78
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,12585.20,11963.17


In [83]:
model_random_forest_cyclical_eval = model_random_forest_cyclical.evaluate(X_test, y_test)
model_random_forest_cyclical_eval

(np.float64(13986.546081124921), np.float64(214076189.93677723))

### Gradient Boosting

In [84]:
start = datetime.datetime.now()
model_gradient_boosting = RegressionTreeMethod(model_type = 'gradientboosting', encoding = 'dummy', n_estimators = 100)
model_gradient_boosting.fit(X_train, y_train)
end = datetime.datetime.now()
model_gradient_boosting_time = end - start
model_gradient_boosting_eval = model_gradient_boosting.evaluate(X_test, y_test)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [85]:
start = datetime.datetime.now()
model_gradient_boosting_cyclical = RegressionTreeMethod(model_type = 'gradientboosting', encoding = 'cyclical', n_estimators = 100)
model_gradient_boosting_cyclical.fit(X_train, y_train)
end = datetime.datetime.now()
model_gradient_boosting_cyclical_time = end - start
print(f"Time elapsed: {model_gradient_boosting_cyclical_time}")

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 34)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


Time elapsed: 0:06:12.873394


In [86]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_gradient_boosting_cyclical.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 46320.38409257429
Diff: 46000.70709257429


In [87]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_gradient_boosting_cyclical.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,46320.38,46000.71
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,57989.86,57178.21
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,62610.82,61996.12
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,62872.41,61761.26
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,52906.97,52100.40
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,64422.67,63317.30
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,58376.97,58086.41
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,62677.72,61922.93
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,63248.91,61772.11
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,62401.01,61778.98


In [88]:
model_gradient_boosting_cyclical_eval = model_gradient_boosting_cyclical.evaluate(X_test, y_test)
model_gradient_boosting_cyclical_eval

(np.float64(51448.02134876383), np.float64(2893497531.2382116))

### Hist Gradient Boosting

In [89]:
start = datetime.datetime.now()
model_hist_gradient_boosting = RegressionTreeMethod(model_type = 'histgradientboosting', encoding = 'dummy', max_iter = 100)
model_hist_gradient_boosting.fit(X_train, y_train)
end = datetime.datetime.now()
model_hist_gradient_boosting_time = end - start
model_hist_gradient_boosting_eval = model_hist_gradient_boosting.evaluate(X_test, y_test)

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 27)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


In [90]:
start = datetime.datetime.now()
model_hist_gradient_boosting_cyclical = RegressionTreeMethod(model_type = 'histgradientboosting', encoding = 'cyclical', max_iter = 100)
model_hist_gradient_boosting_cyclical.fit(X_train, y_train)
end = datetime.datetime.now()
model_hist_gradient_boosting_cyclical_time = end - start
print(f"Time elapsed: {model_hist_gradient_boosting_cyclical_time}")

INFO:root:Train datasets shapes: X: (688815, 19), y: (688815,)
INFO:root:Train datasets columns: Index(['country_code', 'city_code', 'order_id', 'courier_id',
       'activation_timestamp', 'transport', 'pickup_latitude',
       'pickup_longitude', 'delivery_latitude', 'delivery_longitude',
       'time_zone', 'pickup_latitude_rad', 'pickup_longitude_rad',
       'delivery_latitude_rad', 'delivery_longitude_rad',
       'pd_distance_haversine_m', 'pd_distance_haversine_m_sk',
       'pd_distance_manhattan_m', 'velocity'],
      dtype='object')
INFO:root:Starting to encode variables
INFO:root:Encoded dataset shape: X: (688815, 34)
INFO:root:Finished to encode variables. Starting to fit the model
INFO:root:Finished training the model


Time elapsed: 0:00:05.142804


In [91]:
y_test_0 = y_test.loc[0]
y_test_0_pred = model_hist_gradient_boosting_cyclical.predict(X_test.iloc[[0]]).item()
print(f"True delivery time: {y_test_0}")
print(f"Predicted delivery time: {y_test_0_pred}")
print(f"Diff: {y_test_0_pred - y_test_0}")

True delivery time: 319.677
Predicted delivery time: 8132.707823433598
Diff: 7813.030823433598


In [92]:
results_dataset = X_test
results_dataset['y_test'] = y_test
results_dataset['y_test_predicted'] = model_hist_gradient_boosting_cyclical.predict(X_test)
results_dataset['diff'] = results_dataset['y_test_predicted'] - results_dataset['y_test']
results_dataset.head(20)

,country_code,city_code,order_id,courier_id,activation_timestamp,transport,pickup_latitude,pickup_longitude,delivery_latitude,delivery_longitude,...,pickup_latitude_rad,pickup_longitude_rad,delivery_latitude_rad,delivery_longitude_rad,pd_distance_haversine_m,pd_distance_haversine_m_sk,pd_distance_manhattan_m,y_test,y_test_predicted,diff
0,ES,ALC,100933187613,176241347,2024-10-14 23:24:54+00:00,BICYCLE,38.35,-0.48,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,636.15,636.15,893.84,319.68,8132.71,7813.03
1,ES,ALC,100933198445,10191824,2024-10-14 23:37:46+00:00,MOTORBIKE,38.35,-0.48,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,774.11,774.11,1012.03,811.65,11836.07,11024.43
2,ES,ALC,100931749423,169099229,2024-10-14 12:57:38+00:00,CAR,38.36,-0.49,38.34,-0.49,...,0.67,-0.01,0.67,-0.01,2205.90,2205.90,2337.58,614.70,10753.23,10138.53
3,ES,ALC,100931867858,177428955,2024-10-14 13:44:06+00:00,BICYCLE,38.35,-0.47,38.34,-0.50,...,0.67,-0.01,0.67,-0.01,2628.16,2628.16,3677.11,1111.15,11965.78,10854.63
4,ES,ALC,100931917343,3548605,2024-10-14 14:03:20+00:00,CAR,38.35,-0.49,38.35,-0.49,...,0.67,-0.01,0.67,-0.01,681.33,681.33,941.43,806.57,6699.51,5892.95
5,ES,ALC,100931951627,173230053,2024-10-14 14:16:29+00:00,MOTORBIKE,38.36,-0.50,38.37,-0.43,...,0.67,-0.01,0.67,-0.01,6103.51,6103.51,6660.54,1105.37,12674.25,11568.88
6,ES,ALC,100932201108,141248791,2024-10-14 16:08:35+00:00,MOTORBIKE,38.37,-0.51,38.38,-0.50,...,0.67,-0.01,0.67,-0.01,819.69,819.69,1097.23,290.56,7280.13,6989.57
7,ES,ALC,100932473558,39980765,2024-10-14 18:16:45+00:00,CAR,38.35,-0.48,38.35,-0.52,...,0.67,-0.01,0.67,-0.01,2808.36,2808.36,2975.41,754.79,12211.00,11456.21
8,ES,ALC,100932857877,177428955,2024-10-14 20:36:34+00:00,BICYCLE,38.35,-0.47,38.34,-0.48,...,0.67,-0.01,0.67,-0.01,1575.89,1575.89,2230.19,1476.80,11676.75,10199.95
9,ES,ALC,100932926010,171303156,2024-10-14 21:00:05+00:00,MOTORBIKE,38.37,-0.41,38.39,-0.41,...,0.67,-0.01,0.67,-0.01,2062.63,2062.63,2099.82,622.03,10630.60,10008.57


In [93]:
model_hist_gradient_boosting_cyclical_eval = model_hist_gradient_boosting_cyclical.evaluate(X_test, y_test)
model_hist_gradient_boosting_cyclical_eval

(np.float64(7845.208552446852), np.float64(71299830.83477303))

## Models comparison

In [96]:
models = ['model_bl_sum' , 'model_bl_mean', 'model_linear_3d', 'model_linear_7d', 'model_linear', 'model_linear_cyclical',
          'model_linear_minmaxscaler', 'model_linear_stdscaler', 'model_linear_SGD_stdscaler', 'model_decision_tree',
          'model_decision_tree_cyclical', 'model_random_forest', 'model_random_forest_cyclical', 'model_gradient_boosting',
          'model_gradient_boosting_cyclical', 'model_hist_gradient_boosting', 'model_hist_gradient_boosting_cyclical']

data = []
for model in models:
    time_value = eval(model + '_time')
    eval_value = eval(model + '_eval')
    data.append([model, time_value / datetime.timedelta(milliseconds=1), eval_value[0], eval_value[1]])

models_eval = pd.DataFrame(data, columns=['Model', 'Training time [ms]', 'MAE', 'MSE'])
models_eval

,Model,Training time [ms],MAE,MSE
0,model_bl_sum,464.89,256.39,179628.88
1,model_bl_mean,231.85,239.81,157993.40
2,model_linear_3d,234.43,4850.46,23788905.20
3,model_linear_7d,676.09,300.43,198128.50
4,model_linear,1204.04,183.02,114183.02
5,model_linear_cyclical,2297.86,183.29,122189.96
6,model_linear_minmaxscaler,1913.30,183.01,114181.34
7,model_linear_stdscaler,1983.02,183.05,114183.72
8,model_linear_SGD_stdscaler,3635.79,182.88,115406.53
9,model_decision_tree,24373.87,29509.21,1402015056.90
